In [6]:
pip install kaggle

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [14]:
! chmod 600 /root/.kaggle/kaggle.json


In [15]:
!kaggle competitions list

ref                                                                                deadline             category             reward  teamCount  userHasEntered  
---------------------------------------------------------------------------------  -------------------  ---------------  ----------  ---------  --------------  
https://www.kaggle.com/competitions/ai-mathematical-olympiad-prize                 2024-06-27 23:59:00  Featured         $1,048,576        413           False  
https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability        2024-05-27 23:59:00  Featured           $105,000       2976           False  
https://www.kaggle.com/competitions/lmsys-chatbot-arena                            2024-08-05 23:59:00  Research           $100,000        117           False  
https://www.kaggle.com/competitions/learning-agency-lab-automated-essay-scoring-2  2024-07-02 23:59:00  Featured            $50,000       1164           False  
https://www.kaggle.com/competition

In [16]:
!kaggle competitions download -c playground-series-s4e5

 93% 26.0M/28.0M [00:01<00:00, 26.8MB/s]
100% 28.0M/28.0M [00:01<00:00, 16.3MB/s]


In [20]:
!unzip "/content/playground-series-s4e5.zip" -d "/content/drive/MyDrive/CDAC/Machine learning/Kaggle/flood probability/"

Archive:  /content/playground-series-s4e5.zip
  inflating: /content/drive/MyDrive/CDAC/Machine learning/Kaggle/flood probability/sample_submission.csv  
  inflating: /content/drive/MyDrive/CDAC/Machine learning/Kaggle/flood probability/test.csv  
  inflating: /content/drive/MyDrive/CDAC/Machine learning/Kaggle/flood probability/train.csv  


In [21]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.6 MB/s eta 0:00:00


In [23]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import StackingRegressor

In [24]:
train = pd.read_csv(r"/content/drive/MyDrive/CDAC/Machine learning/Kaggle/flood probability/train.csv")
test = pd.read_csv(r"/content/drive/MyDrive/CDAC/Machine learning/Kaggle/flood probability/test.csv")

In [25]:
X = train.drop(["id","FloodProbability"],axis=1)
X_test= test.drop("id",axis=1)
y = train["FloodProbability"]

In [ ]:
#### Stack 1 linear,elastic,xgb ---> lgbm

In [ ]:
elastic = ElasticNet()
lr = LinearRegression()
xgb = XGBRegressor(random_state=24)
lgbm = LGBMRegressor(random_state=24)
stack = StackingRegressor([('LR',lr),('Elastic',elastic),('XGB',xgb)],final_estimator=lgbm,passthrough=True)

In [ ]:
stack.get_params()

In [ ]:
kfold = KFold(n_splits=5,shuffle=True,random_state=24)
params = {"Elastic__l1_ratio":np.arange(0.01,1,5),"Elastic__alpha":np.linspace(0.01,3,5),
          "XGB__learning_rate":np.linspace(0.001,0.9,5),"XGB__max_depth":[None,2,3],
          "XGB__n_estimators":[25,50,100],"final_estimator__learning_rate":np.arange(0.01,1,5),
         "final_estimator__max_depth":[None,2,3,4],"final_estimator__n_estimators":[25,50,100]}
stack1 = GridSearchCV(stack,param_grid=params,cv=kfold,scoring='neg_log_loss',n_jobs=-1)
stack1.fit(X,y)

y_pred  = stack1.predict(X_test)
submit = pd.DataFrame({"id":test["id"],"FloodProbability":y_pred})
submit.to_csv(r"C:\Users\Administrator.DAI-PC2\Downloads\Shubham\Practical Machine Learning\Kaggle\Flood Prediction\stack1.csv",index=False)

In [ ]:
#### Stack 2 xgb,cat,elastic,lgbm ---> lr

In [26]:
elastic = ElasticNet()
lr = LinearRegression()
xgb = XGBRegressor(random_state=24)
cat =  CatBoostRegressor(random_state=24)
lgbm = LGBMRegressor(random_state=24)
stack2 = StackingRegressor([('LGBM',lgbm),('Elastic',elastic),('XGB',xgb),('CAT',cat)],final_estimator=lr,passthrough=True )

stack2.fit(X,y)

y_pred  = stack2.predict(X_test)
submit = pd.DataFrame({"id":test["id"],"FloodProbability":y_pred})
submit.to_csv(r"stack2.csv",index=False)

Streaming output truncated to the last 5000 lines.
4:	learn: 0.0469369	total: 878ms	remaining: 2m 54s
5:	learn: 0.0462008	total: 1.04s	remaining: 2m 52s
6:	learn: 0.0455045	total: 1.2s	remaining: 2m 50s
7:	learn: 0.0448591	total: 1.37s	remaining: 2m 49s
8:	learn: 0.0442060	total: 1.54s	remaining: 2m 49s
9:	learn: 0.0435751	total: 1.71s	remaining: 2m 49s
10:	learn: 0.0429945	total: 1.87s	remaining: 2m 48s
11:	learn: 0.0424257	total: 2.02s	remaining: 2m 46s
12:	learn: 0.0418743	total: 2.17s	remaining: 2m 44s
13:	learn: 0.0413412	total: 2.31s	remaining: 2m 42s
14:	learn: 0.0408185	total: 2.46s	remaining: 2m 41s
15:	learn: 0.0402964	total: 2.63s	remaining: 2m 41s
16:	learn: 0.0397847	total: 2.77s	remaining: 2m 40s
17:	learn: 0.0393182	total: 2.91s	remaining: 2m 38s
18:	learn: 0.0388502	total: 3.06s	remaining: 2m 38s
19:	learn: 0.0383827	total: 3.21s	remaining: 2m 37s
20:	learn: 0.0379587	total: 3.35s	remaining: 2m 36s
21:	learn: 0.0375458	total: 3.49s	remaining: 2m 35s
22:	learn: 0.0371368

In [36]:
!kaggle competitions submit -c playground-series-s4e5 -f stack2.csv -m "StackingRegressor([('LGBM',lgbm),('Elastic',elastic),('XGB',xgb),('CAT',cat)],final_estimator=lr,passthrough=True )"

100% 19.3M/19.3M [00:02<00:00, 8.07MB/s]
400 - Bad Request - Submission not allowed:  Your team has used its daily Submission allowance (5) today, please try again tomorrow UTC (12 hours from now).


In [37]:
! kaggle competitions submissions -c playground-series-s4e5

fileName           date                 description  status    publicScore  privateScore  
-----------------  -------------------  -----------  --------  -----------  ------------  
stack2.csv         2024-05-06 09:50:46               complete  0.82504                    
stack1.csv         2024-05-06 09:46:27               complete  0.86331                    
cat.csv            2024-05-06 06:45:03               complete  0.84602                    
lgbm.csv           2024-05-06 06:42:35               complete  0.76716                    
xgb.csv            2024-05-06 06:41:52               complete  0.81205                    
boot_elastic.csv   2024-05-04 08:56:38               complete  0                          
boot_lr.csv        2024-05-04 08:56:09               complete  0.84458                    
submit_voting.csv  2024-05-04 06:42:41               complete  0.84417                    
submit_voting.csv  2024-05-04 06:40:57               complete  0.84417                    